# Retrieval Augmented Generation

## Setup API clients

In [7]:
import os

import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_ADA_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_FULL_INDEX = "gptkbindex"
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)


## Prepare user question

In [8]:
user_question = "What does a sales manager do?"
user_question_vector = get_embedding(user_question)

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [9]:
r = search_client.search(
        user_question,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields="embedding")],
        query_type="semantic",
        semantic_configuration_name="default")

sources = "\n\n".join([f"[{doc['sourcepage']}]: {doc['content']}\n" for doc in r])

print(sources)

[role_library-23.pdf]: 
Manager of Sales
Job Title: Manager of Sales, Contoso Electronics
Job Summary:
The Manager of Sales will be responsible for leading, managing and motivating the sales team to exceed sales objectives and targets. This person will also be in charge of developing and implementing successful sales strategies, analyzing sales and market data, and continuously evaluating the performance of the sales team and sales processes.
Responsibilities: 


[role_library-24.pdf]: · Lead, manage, and motivate the sales team to ensure that targets and objectives are met.
· Develop and implement successful sales strategies and processes.
· Analyze sales and market data in order to identify trends, opportunities, and areas for
improvement.
· Develop and maintain relationships with key customers and vendors.
· Ensure that all sales activities are conducted in accordance with company policies and procedures.
· Monitor sales team performance and provide regular feedback and coaching.
· 

## Send question and documents to the model

In [10]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

A sales manager is responsible for leading, managing, and motivating a sales team to exceed sales objectives and targets. They develop and implement sales strategies, analyze sales and market data, and evaluate the performance of the sales team. Other responsibilities include developing and maintaining relationships with key customers and vendors, managing the sales budget, and monitoring competitor activity to adjust the sales strategy accordingly [role_library-23.pdf] [role_library-24.pdf].
